In [1]:
import tensorflow as tf
from typing import List, Tuple

2025-04-09 15:49:29.261001: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-09 15:49:29.367898: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-09 15:49:35.312404: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/iscb/wolfson/doririmon/anaconda3/

In [2]:
import sys
sys.path.append('../../../../')

In [3]:
tf.config.set_visible_devices([], 'GPU')

2025-04-09 15:49:48.620986: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2027] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 9.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
2025-04-09 15:49:48.622130: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2027] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 9.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
2025-04-09 15:49:48.623245: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2027] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 9.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
2025-04-09 15:49:48.624366: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2027] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 9.0. CUDA kernels will be jit-compiled from PTX, whi

In [123]:
# Example input
# inputs = tf.keras.Input(shape=(11,))
# x = tf.keras.layers.Dense(1024, activation='relu')(inputs)
# x = TransformerEncoderMLP(hidden_units=(1024, 1024), dropout_rate=0.2)(x)
# model = tf.keras.Model(inputs, x)
# model.summary()

In [122]:
# model(tf.random.uniform((10, 11)))

In [4]:
# def bootstrap_pts_encoder_mlps(hidden_sizes_mlp_a: List[Tuple[int, int]], mlp_a_dropout_rate: float,
#                                hidden_sizes_mlp_c: List[Tuple[int, int]], mlp_c_dropout_rate: float,
#                                activation: str, input_shape: Tuple[int, int],
#                                max_number_of_patches: int) -> tf.keras.models.Model:
#     model = build_model(hidden_sizes_mlp_a, mlp_a_dropout_rate,
#                                          hidden_sizes_mlp_c, mlp_c_dropout_rate,
#                                          activation, input_shape, max_number_of_patches)
#     return model

In [6]:
from model import build_model

In [7]:
build_model(
    hidden_sizes_mlp_a=[(1024, 1024), (1024, 1024)],
    mlp_a_dropout_rate=0.2,
    hidden_sizes_mlp_c=[(1024, 1024), (1024, 1024)],
    mlp_c_dropout_rate=0.2,
    activation='relu',
    input_shape=[10, 9],
    max_number_of_patches=10)

2025-04-09 15:49:58.295170: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(None, 10, 11)


In [5]:
# model = bootstrap_pts_encoder_mlps(
#     hidden_sizes_mlp_a=[(1024, 1024), (1024, 1024)],
#     mlp_a_dropout_rate=0.2,
#     hidden_sizes_mlp_c=[(1024, 1024), (1024, 1024)],
#     mlp_c_dropout_rate=0.2,
#     activation='relu',
#     input_shape=[10, 9],
#     max_number_of_patches=10
# )

In [121]:
# model(tf.random.uniform((20, 10, 9), minval=0, maxval=1),
#       tf.random.uniform((20, 1), minval=0, maxval=1),
#       tf.one_hot(tf.random.uniform((20,), minval=0, maxval=10), depth=11))